In [3]:
#%pip install duckdb leafmap
#%pip install "xarray[io]"
#%pip install jupysql
#%pip install duckdb-engine --quiet # To use DuckDB with JupyterSQL for %%sql commands

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import duckdb
import leafmap
import pandas as pd
import xarray as xr


# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# configurations on jupysql to directly output data to Pandas and to simplify the output
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [6]:
def read_csv_file(file_path):
    df = pd.read_csv(file_path)
    # print(df.head())
    # print(df.tail())
    # print(df.columns)
    # print(df.shape)
    # print(df.info())
    return df

def get_column_values(df, column_name):
    if column_name in df.columns:
        return df[column_name].tolist()
    else:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame")

def process_raster_names(raster_names):
    processed_names = []
    for name in raster_names:
        if ":" in name:
            extracted_name = name.split(":", 1)[1].strip().lower().replace(" ", "_")
            processed_names.append(extracted_name)
    return processed_names

def extract_data_points_vectorized_long_lat(ds, raster_column_values_lowercased):
    # Convert the dataset to a pandas DataFrame
    df = (
        ds.to_dataframe().reset_index()
    )  # .reset_index() is used to convert the index to columns

    # print("---Before reshaping---")
    # print(df.head())
    # print(df.tail())
    # print(df.columns)
    # print(df.shape)
    # print(df.info())

    # Assuming 'df' is your original DataFrame with the columns shown
    df.columns = [
        "raster",
        "latitude",
        "longitude",
        "Basic Demographic Characteristics, v4.10 (2010): Male, Density, 2.5 arc-minutes", #TODO: More generic
    ]

    # Use pivot for vectorized reshaping
    reshaped_df = df.pivot(
        index=["latitude", "longitude"],
        columns="raster",
        values="Basic Demographic Characteristics, v4.10 (2010): Male, Density, 2.5 arc-minutes", #TODO: More generic
    )

    # Reset the index to turn 'latitude' and 'longitude' back into columns
    reshaped_df.reset_index(inplace=True)

    # Rename the raster columns to have a prefix
    reshaped_df.columns = ["latitude", "longitude"] + [
        raster_column_values_lowercased[int(col) - 1] if not pd.isnull(col) else col
        for col in reshaped_df.columns[2:]
    ]

    # print("---After reshaping---")
    # print(reshaped_df.head())
    # print(reshaped_df.tail())
    # print(reshaped_df.columns)
    # print(reshaped_df.shape)
    # print(reshaped_df.info())

    return reshaped_df

def load_dataset(file_path, lat_slice, lon_slice):
    dataset = xr.open_dataset(file_path)
    # TODO: Fix out of memory problem by using dask https://tutorial.xarray.dev/intermediate/xarray_and_dask.html
    data_slice = dataset.sel(latitude=lat_slice, longitude=lon_slice)
    data_slice = data_slice[
        "Basic Demographic Characteristics, v4.10 (2010): Male, Density, 2.5 arc-minutes" #TODO: Make this dynamic
    ]
    return data_slice.compute()

### Paths for data and additional files from original dataset to do lookups/renamings

In [7]:
file_path_male = "../data/gpw_v4_basic_demographic_characteristics_rev11_mt_2010_dens_2pt5_min.nc"
file_path_male_raster_lookup = "../data/gpw_v4_basic_demographic_characteristics_rev11_mt_2010_dens_2pt5_min_lookup.csv"
df_file_path_male_raster_lookup = read_csv_file(file_path_male_raster_lookup)
raster_column_values = get_column_values(
    df_file_path_male_raster_lookup, "raster_name"
)

raster_column_values_lowercased = process_raster_names(raster_column_values)

print(raster_column_values_lowercased)

['male_population_density', 'male_population_density_ages_0_to_4', 'male_population_density_ages_5_to_9', 'male_population_density_ages_10_to_14', 'male_population_density_ages_15_to_19', 'male_population_density_ages_20_to_24', 'male_population_density_ages_25_to_29', 'male_population_density_ages_30_to_34', 'male_population_density_ages_35_to_39', 'male_population_density_ages_40_to_44', 'male_population_density_ages_45_to_49', 'male_population_density_ages_50_to_54', 'male_population_density_ages_55_to_59', 'male_population_density_ages_60_to_64', 'male_population_density_ages_65_and_over', 'data_context', 'mean_administrative_unit_area', 'water_mask', 'land_area', 'water_area', 'national_identifier_grid', 'data_code', 'input_data_year', 'input_data_level', 'input_sex_data_level', 'input_age_data_level', 'growth_rate_start_year', 'growth_rate_end_year', 'growth_rate_administrative_level', 'year_of_most_recent_census']


### Actual loading of data

In [8]:
lat_slice = slice(85, -85)  # Dosen't work with value over 85 or under -85
lon_slice = slice(6, 7)

data_slice_male = load_dataset(file_path_male, lat_slice, lon_slice)
data_slice_male_long_lat = extract_data_points_vectorized_long_lat(data_slice_male, raster_column_values_lowercased)

data_slice_male_long_lat


,latitude,longitude,male_population_density,male_population_density_ages_0_to_4,male_population_density_ages_5_to_9,male_population_density_ages_10_to_14,male_population_density_ages_15_to_19,male_population_density_ages_20_to_24,male_population_density_ages_25_to_29,male_population_density_ages_30_to_34,...,national_identifier_grid,data_code,input_data_year,input_data_level,input_sex_data_level,input_age_data_level,growth_rate_start_year,growth_rate_end_year,growth_rate_administrative_level,year_of_most_recent_census
0,-84.979167,6.020833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-84.979167,6.062500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-84.979167,6.104167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-84.979167,6.145833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-84.979167,6.187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97915,84.979167,6.812500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97916,84.979167,6.854167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97917,84.979167,6.895833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97918,84.979167,6.937500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32767.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_slice_male_long_lat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97920 entries, 0 to 97919
Data columns (total 32 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   latitude                                  97920 non-null  float64
 1   longitude                                 97920 non-null  float64
 2   male_population_density                   27512 non-null  float32
 3   male_population_density_ages_0_to_4       27512 non-null  float32
 4   male_population_density_ages_5_to_9       27512 non-null  float32
 5   male_population_density_ages_10_to_14     27512 non-null  float32
 6   male_population_density_ages_15_to_19     27512 non-null  float32
 7   male_population_density_ages_20_to_24     27512 non-null  float32
 8   male_population_density_ages_25_to_29     27512 non-null  float32
 9   male_population_density_ages_30_to_34     27512 non-null  float32
 10  male_population_density_ages_35_to

In [19]:
con = duckdb.connect() # Note: duckdb.sql connects to the default in-memory database connection
con.install_extension("spatial")
con.load_extension("spatial")
con.sql("CREATE TABLE data_slice_male_long_lat AS SELECT * FROM data_slice_male_long_lat")
con.sql("DESCRIBE data_slice_male_long_lat")

┌───────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│              column_name              │ column_type │  null   │   key   │ default │  extra  │
│                varchar                │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ latitude                              │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ longitude                             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density               │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_0_to_4   │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_5_to_9   │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_10_to_14 │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_15_to_19 

Transform the longitude and latitude field into a geometry type

In [ ]:
sql_query = """
SELECT latitude, longitude, ST_Point(longitude, latitude) AS geom
FROM data_slice_male_long_lat
"""
con.sql(sql_query)

In [ ]:
# Add point geometry 
# Taken from https://motherduck.com/blog/getting-started-gis-duckdb/
con.sql("ALTER TABLE data_slice_male_long_lat ADD COLUMN geom GEOMETRY")
con.sql("UPDATE data_slice_male_long_lat SET geom = ST_Point(longitude, latitude)")
con.sql("DESCRIBE data_slice_male_long_lat")

┌───────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│              column_name              │ column_type │  null   │   key   │ default │  extra  │
│                varchar                │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ latitude                              │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ longitude                             │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density               │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_0_to_4   │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_5_to_9   │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_10_to_14 │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ male_population_density_ages_15_to_19 

In [ ]:
# Do we need to generate a quadkey for every level of zoom? Can we just use the index somehow?
sql_query = """
SELECT geom, 
    ST_QuadKey(geom, 1) as quadkey_z1,
    ST_QuadKey(geom, 2) as quadkey_z2,
    ST_QuadKey(geom, 3) as quadkey_z3,
    ST_QuadKey(geom, 4) as quadkey_z4,
    ST_QuadKey(geom, 5) as quadkey_z5,
    ST_QuadKey(geom, 6) as quadkey_z6,
    ST_QuadKey(geom, 7) as quadkey_z7,
    ST_QuadKey(geom, 8) as quadkey_z8,
    ST_QuadKey(geom, 9) as quadkey_z9,
    ST_QuadKey(geom, 10) as quadkey_z10,
    ST_QuadKey(geom, 11) as quadkey_z11,
    ST_QuadKey(geom, 12) as quadkey_z12,
    ST_QuadKey(geom, 13) as quadkey_z13,
    ST_QuadKey(geom, 14) as quadkey_z14
FROM data_slice_male_long_lat
"""
con.sql(sql_query)


┌──────────────────────────────────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬─────────────┬─────────────┬──────────────┬───────────────┬────────────────┐
│                     geom                     │ quadkey_z1 │ quadkey_z2 │ quadkey_z3 │ quadkey_z4 │ quadkey_z5 │ quadkey_z6 │ quadkey_z7 │ quadkey_z8 │ quadkey_z9 │ quadkey_z10 │ quadkey_z11 │ quadkey_z12  │  quadkey_z13  │  quadkey_z14   │
│                   geometry                   │  varchar   │  varchar   │  varchar   │  varchar   │  varchar   │  varchar   │  varchar   │  varchar   │  varchar   │   varchar   │   varchar   │   varchar    │    varchar    │    varchar     │
├──────────────────────────────────────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼─────────────┼─────────────┼──────────────┼───────────────┼────────────────┤
│ POINT (6.020833333333314 -84.9

In [ ]:
#-- Create an R-tree index on the geometry column
con.sql("CREATE INDEX idx_data_slice_male_long_lat ON data_slice_male_long_lat USING RTREE (geom) WITH (max_node_capacity = 4)")

CatalogException: Catalog Error: Index with name "idx_data_slice_male_long_lat" already exists

In [30]:
con.sql("SELECT * FROM rtree_index_dump('idx_data_slice_male_long_lat')")

┌───────┬──────────────────────────────────────────────────────────────────────────────────┬────────┐
│ level │                                      bounds                                      │ row_id │
│ int32 │                                     box_2df                                      │ int64  │
├───────┼──────────────────────────────────────────────────────────────────────────────────┼────────┤
│     0 │ {'min_x': 6.020833, 'min_y': -84.97917, 'max_x': 6.979167, 'max_y': 30.3125}     │   NULL │
│     1 │ {'min_x': 6.020833, 'min_y': -84.97917, 'max_x': 6.979167, 'max_y': -55.6875}    │   NULL │
│     2 │ {'min_x': 6.020833, 'min_y': -84.97917, 'max_x': 6.979167, 'max_y': -77.02083}   │   NULL │
│     3 │ {'min_x': 6.020833, 'min_y': -84.97917, 'max_x': 6.979167, 'max_y': -82.354164}  │   NULL │
│     4 │ {'min_x': 6.4375, 'min_y': -84.97917, 'max_x': 6.8125, 'max_y': -83.6875}        │   NULL │
│     5 │ {'min_x': 6.4375, 'min_y': -84.97917, 'max_x': 6.5208335, 'max_y': -83.6

In [ ]:
# Taken from https://duckdb.org/docs/extensions/spatial/r-tree_indexes
sql_query = """
SELECT 
  level, 
  bounds::GEOMETRY AS geom, 
  CASE WHEN row_id IS NULL THEN st_area(geom) ELSE NULL END AS area, 
  row_id, 
  CASE WHEN row_id IS NULL THEN 'branch' ELSE 'leaf' END AS kind 
FROM rtree_index_dump('idx_data_slice_male_long_lat') 
ORDER BY area DESC;
"""

con.sql(sql_query)

┌───────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┬────────┬─────────┐
│ level │                                                                                                    geom                                                                                                     │         area          │ row_id │  kind   │
│ int32 │                                                                                                  geometry                                                                                                   │        double         │ int64  │ varchar │
├───────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────